# NN Model I

- Try out Fast.ai tabular_learner

In [1]:
import fastai
from fastai.tabular import *
import sklearn
import sys
import numpy as np
import os
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from kedro.extras.datasets.pandas import CSVDataSet

### Loading Data

In [2]:
BattleResults = catalog.load('Battle_Results_Train')

2020-05-24 17:38:02,016 - kedro.io.data_catalog - INFO - Loading data from `Battle_Results_Train` (CSVDataSet)...


In [3]:
BattleResults.head()

,Name_1,Level_1,HP_1,Attack_1,Defense_1,Sp_Atk_1,Sp_Def_1,Speed_1,Legendary_1,Name_2,Level_2,HP_2,Attack_2,Defense_2,Sp_Atk_2,Sp_Def_2,Speed_2,Legendary_2,WeatherAndTime,BattleResult
0,Psyduck,8,68,65,59,82,63,69,False,Weedle,75,336,192,172,103,103,279,False,Unknown,-336
1,Rapidash,86,691,707,492,571,571,752,False,Zapdos,72,607,523,499,722,523,577,True,Sunshine,133
2,Dratini,93,636,698,484,538,538,538,False,Marowak,73,446,529,720,328,529,284,False,Windy,324
3,Horsea,6,41,47,85,85,29,72,False,Moltres,86,919,886,797,1088,750,797,True,Night,-919
4,Golbat,78,571,519,449,400,481,567,False,Kingler,96,637,1178,1049,456,456,682,False,Unknown,-560


In [4]:
BattleResults.columns

Index(['Name_1', 'Level_1', 'HP_1', 'Attack_1', 'Defense_1', 'Sp_Atk_1',
       'Sp_Def_1', 'Speed_1', 'Legendary_1', 'Name_2', 'Level_2', 'HP_2',
       'Attack_2', 'Defense_2', 'Sp_Atk_2', 'Sp_Def_2', 'Speed_2',
       'Legendary_2', 'WeatherAndTime', 'BattleResult'],
      dtype='object')

### Data Pipeline

In [5]:
data = BattleResults.drop(columns=['Name_1', 'Name_2'])

In [6]:
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

In [7]:
train_data = train_data.iloc[:100000]

In [27]:
def RelativeResult(X):
    HP = X.apply(lambda row: row["HP_1"] if row["BattleResult"] > 0 else row["HP_2"], axis=1).values
    relativized = X["BattleResult"].values / HP
    return relativized

train_data['BattleResult'] = RelativeResult(train_data)

/home/joel/.local/share/virtualenvs/pokethon-gj-dbL-j/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
train_data.head()

,Level_1,HP_1,Attack_1,Defense_1,Sp_Atk_1,Sp_Def_1,Speed_1,Legendary_1,Level_2,HP_2,Attack_2,Defense_2,Sp_Atk_2,Sp_Def_2,Speed_2,Legendary_2,WeatherAndTime,BattleResult
1463814,71,360,650,279,197,595,478,False,78,679,656,458,315,399,285,False,Night,-445
72186,86,675,832,990,389,389,299,False,64,361,320,308,336,416,416,False,Windy,622
2151070,47,163,204,149,191,191,327,False,10,72,57,57,57,57,57,False,Windy,163
1275428,70,538,471,471,471,471,471,False,45,198,132,184,209,209,368,False,Windy,538
744848,96,530,663,416,416,585,1014,False,15,117,127,139,45,45,36,False,Windy,530


### Train

In [14]:
procs= [FillMissing, Categorify, Normalize]
cat_names=['WeatherAndTime', 'Legendary_1', 'Legendary_2']
cont_names=['HP_1', 'Attack_1', 'Defense_1', 'Sp_Atk_1', 'Sp_Def_1', 'Speed_1', 
            'HP_2', 'Attack_2', 'Defense_2', 'Sp_Atk_2', 'Sp_Def_2', 'Speed_2']
x = np.random.permutation(len(train_data))

In [15]:
test = TabularList.from_df(val_data, cat_names=cat_names, cont_names=cont_names, procs=procs)

In [16]:
tabular_data = (TabularList.from_df(train_data, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(x[:300000])
                           .label_from_df(cols='BattleResult', label_cls=FloatList)
                           .add_test(test)
                           .databunch())

In [17]:
tabular_data.show_batch()

StopIteration: 

In [18]:
learn = tabular_learner(tabular_data, layers=[200,100], y_range=[-1,1], metrics=mse)

In [19]:
learn.summary()

TabularModel
Layer (type)         Output Shape         Param #    Trainable 
Embedding            [2]                  2          True      
______________________________________________________________________
Embedding            [2]                  2          True      
______________________________________________________________________
Embedding            [2]                  2          True      
______________________________________________________________________
Dropout              [6]                  0          False     
______________________________________________________________________
BatchNorm1d          [12]                 24         True      
______________________________________________________________________
Linear               [200]                3,800      True      
______________________________________________________________________
ReLU                 [200]                0          False     
_________________________________________________

In [ ]:
learn.

In [34]:
learn.fit(2, 1e-2)

epoch,train_loss,valid_loss,mean_squared_error,time
0,0.101853,0.090937,0.090937,02:30
1,0.107648,0.092120,0.092120,02:32
